In [31]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import math

# 1. Define Tickers and Dates
# -------------------------------
tickers = ['VNQ', 'HYG', 'TIP', 'GOVT', 'LQD', 'IJH', 'SPY', 'VWO', 'EFA']

# Calculate start date as previous 45 trading days from today
start_date = (pd.Timestamp.today() - pd.tseries.offsets.BDay(45)).strftime('%Y-%m-%d')
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

print(f"Downloading data from {start_date} to {end_date}...")

# 2. Download Historical Price Data
# -------------------------------
# Use the 'Close' price for calculations
data = yf.download(tickers, start=start_date, end=end_date)['Close']
data = data.dropna()  # Remove rows with missing data

print("\nHistorical Prices (last 5 rows):")
print(data.tail())

# 3. Compute Daily Returns
# -------------------------------
daily_returns = data.pct_change().dropna()

print("\nDaily Returns (last 5 rows):")
print(daily_returns.tail())

# -------------------------------
# 4. Calculate Performance Metrics
# -------------------------------
metrics = pd.DataFrame(index=tickers)

# a) Average Daily Return & Daily Volatility
metrics['Average Daily Return'] = daily_returns.mean()
metrics['Daily Volatility'] = daily_returns.std()

# b) Annualized Metrics (Assuming 45 trading days)
trading_days = 45
metrics['Annualized Return'] = metrics['Average Daily Return'] * trading_days
metrics['Annualized Volatility'] = metrics['Daily Volatility'] * np.sqrt(trading_days)

# c) Sharpe Ratio
# Assuming an annualized Risk-Free Rate of 2% (0.02)
risk_free_rate = 0.02
metrics['Sharpe Ratio'] = (metrics['Annualized Return'] - risk_free_rate) / metrics['Annualized Volatility']

# -------------------------------
# 5. Compute Information Ratio
# -------------------------------
# Download benchmark data (S&P 500) and compute its daily returns.
benchmark_ticker = '^GSPC'
benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date)['Close']
benchmark_data = benchmark_data.dropna()
benchmark_returns = benchmark_data.pct_change().dropna()

info_ratios = {}
for ticker in tickers:
    # Align security and benchmark returns on common dates
    combined = pd.concat([daily_returns[ticker], benchmark_returns], axis=1, join='inner')
    combined.columns = ['security', 'benchmark']
    # Calculate excess returns (security return minus benchmark return)
    excess_returns = combined['security'] - combined['benchmark']

    # Annualized excess return and tracking error (volatility of excess returns)
    mean_excess_return = excess_returns.mean() * trading_days
    tracking_error = excess_returns.std() * np.sqrt(trading_days)

    info_ratios[ticker] = mean_excess_return / tracking_error if tracking_error != 0 else np.nan

metrics['Information Ratio'] = pd.Series(info_ratios)

# -------------------------------
# 6. Compute Treynor Ratio
# -------------------------------
# For Treynor Ratio, compute Beta for each security relative to the benchmark.
betas = {}
PerformanceMetrics = {}
for ticker in tickers:
    # Align returns for covariance calculation
    combined = pd.concat([daily_returns[ticker], benchmark_returns], axis=1, join='inner')
    combined.columns = ['security', 'benchmark']

    # Calculate covariance and variance
    cov = np.cov(combined['security'], combined['benchmark'])[0, 1]
    var_benchmark = np.var(combined['benchmark'])
    betas[ticker] = cov / var_benchmark if var_benchmark != 0 else np.nan

# Treynor Ratio: (Annualized Return - Risk-Free Rate) / Beta
treynor_ratios = {}
for ticker in tickers:
    beta = betas[ticker]
    if beta and beta != 0:
        treynor_ratios[ticker] = (metrics.loc[ticker, 'Annualized Return'] - risk_free_rate) / beta
    else:
        treynor_ratios[ticker] = np.nan

metrics['Treynor Ratio'] = pd.Series(treynor_ratios)

# -------------------------------
# 7. Print Final Metrics
# -------------------------------
print("\nPerformance Metrics for each security:")
print(metrics)
print(pd.Series(PerformanceMetrics, name='Performance Metrics'))

print("\nBeta Values:")
print(pd.Series(betas, name='Beta'))

[*********************100%***********************]  9 of 9 completed
[*********************100%***********************]  1 of 1 completed


Historical Prices (last 5 rows):
Ticker            EFA       GOVT        HYG        IJH         LQD  \
Date                                                                 
2025-03-27  83.199997  22.719618  78.557762  59.279999  107.416306   
2025-03-28  82.459999  22.859186  78.418442  58.209999  108.063812   
2025-03-31  81.730003  22.919001  78.508003  58.349998  108.273003   
2025-04-01  81.849998  22.969999  78.690002  58.660000  108.660004   
2025-04-02  82.040001  22.950001  78.849998  59.599998  108.910004   

Ticker             SPY         TIP        VNQ        VWO  
Date                                                      
2025-03-27  567.080017  109.387375  89.830002  46.090000  
2025-03-28  555.659973  109.943810  89.709999  45.310001  
2025-03-31  559.390015  110.380997  90.540001  45.259998  
2025-04-01  560.969971  110.449997  90.580002  45.410000  
2025-04-02  564.520020  110.279999  91.129997  45.450001  

Daily Returns (last 5 rows):
Ticker           EFA      GOVT  

# Component metrics

metrics

In [33]:
portfolio_stats = {
    "Average Daily Return": metrics["Average Daily Return"].mean(),
    "Average Daily Std Dev (Volatility)": metrics["Daily Volatility"].mean(),
    "Average Annualized Return": metrics["Annualized Return"].mean(),
    "Average Annualized Std Dev": metrics["Annualized Volatility"].mean(),
    "Average Sharpe Ratio": metrics["Sharpe Ratio"].mean(),
    "Average Information Ratio": metrics["Information Ratio"].mean(),
    "Average Treynor Ratio": metrics["Treynor Ratio"].mean()
}

# Portfolio metrics

In [34]:
Portfolio = pd.DataFrame(portfolio_stats,index=["Portfolio Stats"])
Portfolio

,Average Daily Return,Average Daily Std Dev (Volatility),Average Annualized Return,Average Annualized Std Dev,Average Sharpe Ratio,Average Information Ratio,Average Treynor Ratio
Portfolio Stats,-0.000033,0.007084,-0.00148,0.047523,-0.3025,1.036265,-0.053981
